In [ ]:
import sys
from pathlib import Path

# Add the 'backtest' directory to the system path
notebook_dir = Path().resolve()
backtest_dir = notebook_dir.parent / 'backtest'
sys.path.append(str(backtest_dir))

In [ ]:
import pandas as pd
from backtest import Backtest, Strategy, Order

In [ ]:
# First run the get_data.ipynb notebook to generate the data file
# or use the code from get_data.ipynb here to download the data directly

# Read the OHLCV data from data/BTCUSDT.csv
# Use small subset of data for easier analysis
data = pd.read_csv('data/BTCUSDT.csv', index_col='Date', parse_dates=True).loc['2025-01-01':]

In [ ]:
class LimitOrderStrategy(Strategy):
    """
    A strategy that places a limit order to buy at a specific price
    """

    def __init__(self):
        self.buy_date = pd.Timestamp("2025-02-01")
        self.buy_price = 85000.0

    def on_candle(self, historical_data, positions_book):
        if historical_data.index[-1] == self.buy_date:
            return [Order(action="enter", value=10000.0, price=self.buy_price)]
        return []

In [ ]:
backtest = Backtest(data, LimitOrderStrategy, equity=20000.0)

backtest.run()

In [ ]:
stats = backtest.stats()

In [ ]:
# Entry series using actual entry prices
entry_series = pd.Series(
    {pos.entry_time: pos.entry_price for pos in backtest.positions}
)

# Create a series of "Open" prices at exit dates from backtest.pnl_df
exited_positions = [pos for pos in backtest.positions if pos.exit_time is not None]
# Exit series using actual exit prices
exit_series = pd.Series(
    {pos.exit_time: pos.exit_price for pos in exited_positions}
)


In [ ]:
from matplotlib import pyplot as plt

df = backtest.pnl_df

plt.figure(figsize=(14, 6))
plt.plot(df['Close'], label="Close Price", alpha=0.7)

plt.scatter(entry_series.index, entry_series, marker='^', color="green", label="Buy", s=100)

plt.scatter(exit_series.index, exit_series, marker='v', color="red", label="Sell", s=100)

for idx, price in entry_series.items():
    plt.annotate(idx.strftime('%Y-%m-%d'), xy=(idx, price), xytext=(0, 10), 
                 textcoords='offset points', ha='center', fontsize=8, color='green')
    
for idx, price in exit_series.items():
    plt.annotate(idx.strftime('%Y-%m-%d'), xy=(idx, price), xytext=(0, -15), 
                 textcoords='offset points', ha='center', fontsize=8, color='red')

plt.title("BTC/USDT with Buy/Sell Signals")
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True)
plt.show()

# Plot the Total PnL
plt.figure(figsize=(14, 2))
plt.plot(df['total_pnl'], label="Total PnL", alpha=0.7)
plt.title("Total PnL Over Time")
plt.xlabel('Date')
plt.ylabel('Total PnL (USDT)')
plt.legend()
plt.grid(True)
plt.show()